# Export to XLSX (Excel)

In [1]:
# import necessary libraries
import pandas as pd
import datetime as dt
from helpers.helper_functions import init_connection, get_all_labels

# run the initialization notebook to connect to the neo4j database
graph = init_connection()

Connected to the database


In [4]:
# define function to export data to Excel
def export_to_excel():
    current_time = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    # define the file path where the Excel file will be saved
    file_path = f"data/export/export_{current_time}.xlsx"
    # create a Pandas Excel writer using Openpyxl as the engine
    writer = pd.ExcelWriter(file_path, engine='openpyxl')
    # get all labels from the graph
    labels = get_all_labels(graph=graph)

    # iterate over all labels 
    for label in labels:
        # retrieve all nodes with the current label
        nodes_result = graph.run(f"MATCH (n:{label}) RETURN properties(n) AS props")
        
        rows = [record["props"] for record in nodes_result]
        if rows:  # only write if rows exists
            # create a DataFrame from the nodes result
            nodes_df = pd.DataFrame(rows)
            # write the DataFrame to the Excel file
            nodes_df.to_excel(writer, sheet_name=label[:31], index=False)  # Excel Sheet-Namen dürfen max 31 Zeichen haben

        # retrieve all relationships for the current label
        rels_result = graph.run(f"MATCH (n:{label})-[r]->(m) RETURN n.name as start_node_name, LABELS(m)[0] as end_node_label, m.name as end_node_name, TYPE(r) as rel_type")

        # create a DataFrame from the relationships result
        rels_df = pd.DataFrame(rels_result.data())

        if not rels_df.empty:  # only create a worksheet if rels_df is not empty
            rels_df.to_excel(writer, sheet_name=f"REL_{label[:31]}", index=False)
    
    # close the Pandas Excel writer and save the file
    writer.close()

    print("Export completed. File saved at:", file_path)

In [5]:
export_to_excel()

Export completed. File saved at: data/export/xlsx/export_2025-04-16_14-41-21.xlsx
